修改路径
将顶部 hr_dir、sr_dir 改为你的高分辨率（Ground‐Truth）和超分结果目录。

In [3]:
import os
import cv2
import numpy as np
import lpips
import torch
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy import ndimage
from PIL import Image
from torchvision.models import inception_v3
from torch.nn.functional import adaptive_avg_pool2d
from scipy.linalg import sqrtm
import os
print("当前工作目录：", os.getcwd())

class SARImageEvaluator:
    def __init__(self, real_dir, fake_dir, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        初始化SAR图像评估器
        :param real_dir: 参考图像（真实高分辨率）文件夹路径
        :param fake_dir: 生成图像文件夹路径
        :param device: 计算设备 (cuda/cpu)
        """
        print(real_dir)
        print(fake_dir)
        self.real_dir = real_dir
        self.fake_dir = fake_dir
        self.device = device
        
        # 确保文件夹存在
        assert os.path.exists(self.real_dir), f"真实图像文件夹不存在: {self.real_dir}"
        assert os.path.exists(self.fake_dir), f"生成图像文件夹不存在: {self.fake_dir}"
        
        # 获取匹配的文件列表（按文件名排序）
        self.real_files = sorted([f for f in os.listdir(real_dir) if f.endswith(('.png', '.jpg', '.tif'))])
        self.fake_files = sorted([f for f in os.listdir(fake_dir) if f.endswith(('.png', '.jpg', '.tif'))])
        
        # 检查文件数量是否匹配
        assert len(self.real_files) == len(self.fake_files), \
            f"文件数量不匹配: 真实图像 {len(self.real_files)} 张, 生成图像 {len(self.fake_files)} 张"
        
        # 初始化LPIPS模型
        self.lpips_model = lpips.LPIPS(net='alex').to(device)
        self.lpips_model.eval()
        
        # 初始化FID模型
        self.inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
        self.inception_model.eval()

    def load_image(self, path, convert_to_single_channel=True):
        """
        加载图像并进行预处理
        :param path: 图像路径
        :param convert_to_single_channel: 是否转换为单通道
        :return: 预处理后的图像 (0-1范围)
        """
        # 加载图像 (保留原始数据类型)
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        
        # 处理16位图像
        if img.dtype == np.uint16:
            img = img.astype(np.float32) / 65535.0
        # 处理8位图像
        elif img.dtype == np.uint8:
            img = img.astype(np.float32) / 255.0
        # 浮点图像直接使用
        elif img.dtype == np.float32 or img.dtype == np.float64:
            pass
        else:
            raise ValueError(f"不支持的图像数据类型: {img.dtype}")
        
        # 转换为单通道（如果指定且图像是多通道）
        if convert_to_single_channel and len(img.shape) == 3:
            # 对于SAR图像，通常使用平均值或第一个通道
            img = np.mean(img, axis=2) if img.shape[2] > 1 else img[:, :, 0]
        
        return img

    def calculate_psnr(self, real, fake):
        """计算PSNR (峰值信噪比)"""
        # 确保图像在0-1范围内
        real = np.clip(real, 0, 1)
        fake = np.clip(fake, 0, 1)
        
        # 对于单通道SAR图像，使用最大值为1
        data_range = 1.0
        return psnr(real, fake, data_range=data_range)

    def calculate_ssim(self, real, fake):
        """计算SSIM (结构相似性)"""
        # 确保图像在0-1范围内
        real = np.clip(real, 0, 1)
        fake = np.clip(fake, 0, 1)
        
        # 对于单通道图像
        win_size = min(7, min(real.shape) - 1)  # 确保窗口大小有效
        return ssim(real, fake, win_size=win_size, data_range=1.0)

    def calculate_lpips(self, real_path, fake_path):
        """
        计算LPIPS (学习感知图像块相似度)
        注意：需要3通道RGB输入
        """
        # 加载图像（不转换为单通道）
        real = self.load_image(real_path, convert_to_single_channel=False)
        fake = self.load_image(fake_path, convert_to_single_channel=False)
        
        # 处理通道数
        if len(real.shape) == 2:
            real = np.stack([real]*3, axis=-1)
        elif real.shape[2] == 1:
            real = np.repeat(real, 3, axis=2)
        elif real.shape[2] == 4:  # 处理RGBA图像
            real = real[:, :, :3]
        
        if len(fake.shape) == 2:
            fake = np.stack([fake]*3, axis=-1)
        elif fake.shape[2] == 1:
            fake = np.repeat(fake, 3, axis=2)
        elif fake.shape[2] == 4:
            fake = fake[:, :, :3]
        
        # 转换为PyTorch张量 (HWC -> CHW)
        real_tensor = torch.tensor(real).permute(2, 0, 1).unsqueeze(0).float().to(self.device)
        fake_tensor = torch.tensor(fake).permute(2, 0, 1).unsqueeze(0).float().to(self.device)
        
        # 计算LPIPS
        with torch.no_grad():
            return self.lpips_model(real_tensor * 2 - 1, fake_tensor * 2 - 1).item()

    def calculate_enl(self, img, region_size=200):
        """
        计算ENL (等效视数) - 仅适用于SAR图像
        :param img: 单通道SAR图像 (强度图像)
        :param region_size: 用于计算的均匀区域大小
        """
        # 选择图像中心的均匀区域
        h, w = img.shape
        y_start = max(0, h//2 - region_size//2)
        x_start = max(0, w//2 - region_size//2)
        y_end = min(h, y_start + region_size)
        x_end = min(w, x_start + region_size)
        
        # 提取均匀区域
        uniform_region = img[y_start:y_end, x_start:x_end]
        
        # 计算均值和方差
        mean_val = np.mean(uniform_region)
        var_val = np.var(uniform_region)
        
        # 避免除以零
        if var_val < 1e-10:
            return 0.0
        
        # ENL公式: (mean)^2 / variance
        return (mean_val ** 2) / var_val

    def calculate_epi(self, real, fake):
        """
        计算EPI (边缘保持指数)
        :param real: 参考图像
        :param fake: 生成图像
        """
        # 计算梯度 (使用Sobel算子)
        sobel_real_x = ndimage.sobel(real, axis=0)
        sobel_real_y = ndimage.sobel(real, axis=1)
        grad_real = np.sqrt(sobel_real_x**2 + sobel_real_y**2)
        
        sobel_fake_x = ndimage.sobel(fake, axis=0)
        sobel_fake_y = ndimage.sobel(fake, axis=1)
        grad_fake = np.sqrt(sobel_fake_x**2 + sobel_fake_y**2)
        
        # 计算梯度差异
        grad_diff = np.abs(grad_real - grad_fake)
        
        # 避免除以零
        if np.sum(grad_real) < 1e-10:
            return 0.0
        
        # EPI公式: 1 - (|∇HR - ∇SR| / |∇HR|)
        return 1 - np.sum(grad_diff) / np.sum(grad_real)

    def get_inception_features(self, img_tensor):
        """获取Inception v3特征 (用于FID计算)"""
        with torch.no_grad():
            # 调整图像大小以适应Inception输入
            if img_tensor.shape[2] != 299 or img_tensor.shape[3] != 299:
                img_tensor = torch.nn.functional.interpolate(
                    img_tensor, size=(299, 299), mode='bilinear', align_corners=False)
            
            # 获取特征
            features = self.inception_model(img_tensor)
            
            # 处理辅助输出
            if features.dim() > 2:
                features = adaptive_avg_pool2d(features, (1, 1))
                features = features.view(features.size(0), -1)
            
            return features.squeeze().cpu().numpy()

    def calculate_fid(self):
        """计算FID (Fréchet Inception Distance)"""
        real_features = []
        fake_features = []
        
        for real_file, fake_file in zip(self.real_files, self.fake_files):
            real_path = os.path.join(self.real_dir, real_file)
            fake_path = os.path.join(self.fake_dir, fake_file)
            
            # 加载图像 (不转换为单通道)
            real = self.load_image(real_path, convert_to_single_channel=False)
            fake = self.load_image(fake_path, convert_to_single_channel=False)
            
            # 处理通道数
            if len(real.shape) == 2:
                real = np.stack([real]*3, axis=-1)
            elif real.shape[2] == 1:
                real = np.repeat(real, 3, axis=2)
            elif real.shape[2] == 4:
                real = real[:, :, :3]
            
            if len(fake.shape) == 2:
                fake = np.stack([fake]*3, axis=-1)
            elif fake.shape[2] == 1:
                fake = np.repeat(fake, 3, axis=2)
            elif fake.shape[2] == 4:
                fake = fake[:, :, :3]
            
            # 转换为PyTorch张量 (HWC -> CHW)
            real_tensor = torch.tensor(real).permute(2, 0, 1).unsqueeze(0).float().to(self.device)
            fake_tensor = torch.tensor(fake).permute(2, 0, 1).unsqueeze(0).float().to(self.device)
            
            # 获取特征
            real_features.append(self.get_inception_features(real_tensor))
            fake_features.append(self.get_inception_features(fake_tensor))
        
        # 转换为numpy数组
        real_features = np.array(real_features)
        fake_features = np.array(fake_features)
        
        # 计算均值和协方差
        mu_real = np.mean(real_features, axis=0)
        sigma_real = np.cov(real_features, rowvar=False)
        
        mu_fake = np.mean(fake_features, axis=0)
        sigma_fake = np.cov(fake_features, rowvar=False)
        
        # 计算FID
        diff = mu_real - mu_fake
        cov_mean = sqrtm(sigma_real.dot(sigma_fake))
        
        # 处理复数结果
        if np.iscomplexobj(cov_mean):
            cov_mean = cov_mean.real
        
        fid = diff.dot(diff) + np.trace(sigma_real + sigma_fake - 2 * cov_mean)
        return fid

    def evaluate_all(self):
        """计算所有指标"""
        results = {
            'PSNR': [],
            'SSIM': [],
            'LPIPS': [],
            'ENL_real': [],
            'ENL_fake': [],
            'EPI': []
        }
        
        # 逐对计算指标
        for real_file, fake_file in zip(self.real_files, self.fake_files):
            real_path = os.path.join(self.real_dir, real_file)
            fake_path = os.path.join(self.fake_dir, fake_file)
            
            # 加载图像 (转换为单通道)
            real = self.load_image(real_path, convert_to_single_channel=True)
            fake = self.load_image(fake_path, convert_to_single_channel=True)
            
            # 计算PSNR
            results['PSNR'].append(self.calculate_psnr(real, fake))
            
            # 计算SSIM
            results['SSIM'].append(self.calculate_ssim(real, fake))
            
            # 计算LPIPS (需要原始图像路径)
            results['LPIPS'].append(self.calculate_lpips(real_path, fake_path))
            
            # 计算ENL (真实和生成图像)
            results['ENL_real'].append(self.calculate_enl(real))
            results['ENL_fake'].append(self.calculate_enl(fake))
            
            # 计算EPI
            results['EPI'].append(self.calculate_epi(real, fake))
        
        # 计算FID (整个数据集)
        fid = self.calculate_fid()
        
        # 计算平均值
        avg_results = {k: np.mean(v) for k, v in results.items()}
        avg_results['FID'] = fid
        
        # 返回详细结果和平均值
        return {
            'per_image': results,
            'average': avg_results
        }

    def save_results(self, results, output_file='sar_evaluation_results.txt'):
        """保存评估结果到文件"""
        with open(output_file, 'w') as f:
            f.write("SAR Image Super-Resolution Evaluation Results\n")
            f.write("=" * 60 + "\n\n")
            
            f.write(f"Real Images Directory: {self.real_dir}\n")
            f.write(f"Fake Images Directory: {self.fake_dir}\n")
            f.write(f"Number of Images: {len(self.real_files)}\n\n")
            
            f.write("Per-Image Metrics:\n")
            f.write("-" * 60 + "\n")
            f.write("Filename\tPSNR(dB)\tSSIM\tLPIPS\tENL_real\tENL_fake\tEPI\n")
            
            for i, (real_file, fake_file) in enumerate(zip(self.real_files, self.fake_files)):
                f.write(f"{real_file}\t")
                f.write(f"{results['per_image']['PSNR'][i]:.4f}\t")
                f.write(f"{results['per_image']['SSIM'][i]:.4f}\t")
                f.write(f"{results['per_image']['LPIPS'][i]:.4f}\t")
                f.write(f"{results['per_image']['ENL_real'][i]:.4f}\t")
                f.write(f"{results['per_image']['ENL_fake'][i]:.4f}\t")
                f.write(f"{results['per_image']['EPI'][i]:.4f}\n")
            
            f.write("\nAverage Metrics:\n")
            f.write("-" * 60 + "\n")
            f.write(f"PSNR: {results['average']['PSNR']:.4f} dB\n")
            f.write(f"SSIM: {results['average']['SSIM']:.4f}\n")
            f.write(f"LPIPS: {results['average']['LPIPS']:.4f} (lower is better)\n")
            f.write(f"ENL (Real): {results['average']['ENL_real']:.4f}\n")
            f.write(f"ENL (Fake): {results['average']['ENL_fake']:.4f}\n")
            f.write(f"EPI: {results['average']['EPI']:.4f}\n")
            f.write(f"FID: {results['average']['FID']:.4f} (lower is better)\n")
            
            f.write("\n" + "=" * 60 + "\n")
            f.write("Note:\n")
            f.write("- PSNR: Peak Signal-to-Noise Ratio\n")
            f.write("- SSIM: Structural Similarity Index\n")
            f.write("- LPIPS: Learned Perceptual Image Patch Similarity\n")
            f.write("- ENL: Equivalent Number of Looks (SAR specific)\n")
            f.write("- EPI: Edge Preservation Index\n")
            f.write("- FID: Fréchet Inception Distance\n")

        print(f"Results saved to {output_file}")

# 使用示例
if __name__ == "__main__":
    # 设置路径
    REAL_DIR = "D:\Desktop\workspace\workspace\edsr\GF3_test_sahndong_HR"  # 替换为真实图像文件夹路径
    FAKE_DIR = "D:\Desktop\workspace\workspace\ScSR_3\\all_SR"  # 替换为生成图像文件夹路径
    if not os.path.exists(FAKE_DIR) and os.path.exists(REAL_DIR):
        raise ValueError(f"REAL:{os.path.exists(REAL_DIR)};FAKE:{os.path.exists(FAKE_DIR)}")
    # 初始化评估器
    evaluator = SARImageEvaluator(REAL_DIR, FAKE_DIR)
    
    # 执行评估
    results = evaluator.evaluate_all()
    
    # 打印平均结果
    print("\nAverage Evaluation Results:")
    print("=" * 50)
    print(f"PSNR: {results['average']['PSNR']:.4f} dB")
    print(f"SSIM: {results['average']['SSIM']:.4f}")
    print(f"LPIPS: {results['average']['LPIPS']:.4f}")
    print(f"ENL (Real): {results['average']['ENL_real']:.4f}")
    print(f"ENL (Fake): {results['average']['ENL_fake']:.4f}")
    print(f"EPI: {results['average']['EPI']:.4f}")
    print(f"FID: {results['average']['FID']:.4f}")
    
    # 保存结果到文件
    evaluator.save_results(results)

<>:355: SyntaxWarning: invalid escape sequence '\D'
<>:356: SyntaxWarning: invalid escape sequence '\D'
<>:355: SyntaxWarning: invalid escape sequence '\D'
<>:356: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Admin\AppData\Local\Temp\ipykernel_9724\19630762.py:355: SyntaxWarning: invalid escape sequence '\D'
  REAL_DIR = "D:\Desktop\workspace\workspace\edsr\GF3_test_sahndong_HR"  # 替换为真实图像文件夹路径
C:\Users\Admin\AppData\Local\Temp\ipykernel_9724\19630762.py:356: SyntaxWarning: invalid escape sequence '\D'
  FAKE_DIR = "D:\Desktop\workspace\workspace\ScSR_3\\all_SR"  # 替换为生成图像文件夹路径


当前工作目录： d:\Desktop\workspace\workspace\edsr
D:\Desktop\workspace\workspace\edsr\GF3_test_sahndong_HR
D:\Desktop\workspace\workspace\ScSR_3\all_SR
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: d:\download_applications\Anaconda3\envs\common\Lib\site-packages\lpips\weights\v0.1\alex.pth

Average Evaluation Results:
PSNR: 17.6402 dB
SSIM: 0.5557
LPIPS: 0.3693
ENL (Real): 2.4245
ENL (Fake): 11.1989
EPI: 0.4008
FID: 637.4554
Results saved to sar_evaluation_results.txt


In [ ]:
# 设置路径
REAL_DIR = "test_imgs_LR"  # 替换为真实图像文件夹路径
FAKE_DIR = "test_imgs_sr"  # 替换为生成图像文件夹路径
if not (os.path.exists(FAKE_DIR) and os.path.exists(REAL_DIR)):
    raise ValueError(f"REAL:{os.path.exists(REAL_DIR)};FAKE:{os.path.exists(FAKE_DIR)}")
# 初始化评估器
evaluator = SARImageEvaluator(os.path.abspath(REAL_DIR), os.path.abspath(FAKE_DIR))

# 执行评估
results = evaluator.evaluate_all()

# 打印平均结果
print("\nAverage Evaluation Results:")
print("=" * 50)
print(f"PSNR: {results['average']['PSNR']:.4f} dB")
print(f"SSIM: {results['average']['SSIM']:.4f}")
print(f"LPIPS: {results['average']['LPIPS']:.4f}")
print(f"ENL (Real): {results['average']['ENL_real']:.4f}")
print(f"ENL (Fake): {results['average']['ENL_fake']:.4f}")
print(f"EPI: {results['average']['EPI']:.4f}")
print(f"FID: {results['average']['FID']:.4f}")

# 保存结果到文件
evaluator.save_results(results)

: 

In [14]:
real_imgs=os.listdir(REAL_DIR)
r_img=cv2.imread(os.path.join(REAL_DIR,real_imgs[0]),cv2.IMREAD_UNCHANGED)
r_img.shape

(128, 128)

In [15]:
fake_imgs=os.listdir(FAKE_DIR)
r_img=cv2.imread(os.path.join(FAKE_DIR,fake_imgs[0]),cv2.IMREAD_GRAYSCALE)
r_img.shape

(512, 512)

In [2]:
import numpy, torch, cv2
print("OK:", numpy.__version__, torch.__version__)

import torch
print(torch.__version__)
print("CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())


OK: 2.3.2 2.8.0+cu126
2.8.0+cu126
CUDA: 12.6
CUDA available: True
